In [1]:
#IMPORT PACKAGES
import pandas as pd
import csv
import numpy as np
from tqdm.notebook import tqdm

In [2]:
#MOUNT GOOGLE DRIVE WITH RESULTS FROM CELL PROFILER
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#CELL PROFILER OUTPUT FILEPATH
fpath1 = "/content/drive/MyDrive/Results/220412/3D42/MyExpt_ApoptosisSignal.txt"
fpath2 = "/content/drive/MyDrive/Results/220412/3D42/MyExpt_Object relationships.txt"
fpath3 = "/content/drive/MyDrive/Results/220412/3D42/MyExpt_TransfectedCells.txt"
fpath4 = "/content/drive/MyDrive/Results/220412/3D42/MyExpt_Cell_area_assignment.txt"

#SCRIPT OUTPUT PATH
outpath1 = "/content/drive/MyDrive/Results/220412/3D42/outresults_by_cell.csv"
outpath2 = "/content/drive/MyDrive/Results/220412/3D42/results_by_well_TFd.csv"
outpath3 = "/content/drive/MyDrive/Results/220412/3D42/results_by_well_all.csv"

IMPORT DATA INTO DATAFRAME

In [4]:
ApoptosisSig_df = pd.read_csv(fpath1, sep='\t')

In [5]:
ObjRelate_df = pd.read_csv(fpath2, sep='\t')

In [6]:
TFcells_df = pd.read_csv(fpath3, sep='\t')

In [7]:
AreaAssign_df = pd.read_csv(fpath4, sep='\t')

In [8]:
#TRANSLATE APOPTOSIS OUTPUT FILE INTO DICTIONARY WITH APOPTOSIS SIGNAL 
#AREA AND INTENSITY PER UNIQUE OBJECT 
#AND
#TRANSLATE IMAGE NUMBER INTO WELL NUMBER

ApoptosisObjects_dict = {}

imgnum_to_well = {}

for row_dict in ApoptosisSig_df.to_dict(orient="records"):
    well=row_dict['Metadata_Well']
    objnum=well+"_"+str(row_dict['ObjectNumber'])
    intensity=row_dict['Intensity_IntegratedIntensity_mCh']
    area=row_dict['AreaShape_Area']
    
    ApoptosisObjects_dict[objnum] = {}

    ApoptosisObjects_dict[objnum]["intensity"] = intensity
    ApoptosisObjects_dict[objnum]["area"] = area
    
    if row_dict['ImageNumber'] not in imgnum_to_well:
        imgnum_to_well[row_dict['ImageNumber']]=well
    else:
        assert imgnum_to_well[row_dict['ImageNumber']]==well

In [9]:
#TRANSLATE TRANSFECTION SIGNAL FILE INTO DICTIONARY WITH
#AREA AND INTENSITY PER UNIQUE OBJECT 
#AND
#COMPLETE/ASSERT TRANSLATION IMAGE NUMBER INTO WELL NUMBER

TFObjects_dict = {}

for row_dict in TFcells_df.to_dict(orient="records"):
    well=row_dict['Metadata_Well']
    objnum=well+"_"+str(row_dict['ObjectNumber'])
    intensity=row_dict['Intensity_IntegratedIntensity_GFP']
    area=row_dict['AreaShape_Area']
    
    TFObjects_dict[objnum] = {}

    TFObjects_dict[objnum]["intensity"] = intensity
    TFObjects_dict[objnum]["area"] = area
    
    if row_dict['ImageNumber'] not in imgnum_to_well:
        imgnum_to_well[row_dict['ImageNumber']]=well
    else:
        assert imgnum_to_well[row_dict['ImageNumber']]==well

In [10]:
#TRANSLATE DAPI CELL AREA ASSIGNMENT INTO DICTIONARY WITH 
#ALL CELLS PER WELL AND OPTIONS TO ASSIGN TRANSFECTION
#AND APOPTOSIS SIGNAL VALUES TO THEM

Cell_dict = {}

for row_dict in AreaAssign_df.to_dict(orient="records"):
    
    wellid=row_dict['Metadata_Well']
    cell_obj_id = str(row_dict['ObjectNumber'])
    cellnum=wellid+"_"+cell_obj_id
    
    assert cellnum not in Cell_dict
    Cell_dict[cellnum] = {}
    Cell_dict[cellnum]["Red_cytoplasm"] = {}
    Cell_dict[cellnum]["Red_cytoplasm"]["list of vals"] = []
    Cell_dict[cellnum]["Transfected"] = {}
    Cell_dict[cellnum]["Transfected"]["list of vals"] = []

In [11]:
#ASSIGN APOPTOSIS SIGNAL AND TRANSFECITON SIGNAL TO CORRECT CELL USING THE RELATIONS OUTPUT
#FILL CELL DICTIONARY WITH APOPTOSIS AND TRANSFECTION SIGNAL PER CELL
done = set()
pbar=tqdm(ObjRelate_df.to_dict(orient="records"))
for row_dict in pbar:
    
    assert row_dict['First Image Number'] == row_dict['Second Image Number']
    image_ID=row_dict['First Image Number']
    
    if row_dict['First Object Name'] == "Cell_area_assignment":
        #cellnum=str(row_dict['First Image Number'])+"_"+str(row_dict['First Object Number'])
        #objnum=str(row_dict['Second Image Number'])+"_"+str(row_dict['Second Object Number'])
        
        cell_key_word="First"
        obj_key_word="Second"
        objtype=row_dict['Second Object Name']
    elif row_dict['Second Object Name'] == "Cell_area_assignment":
        #cellnum=str(row_dict['Second Image Number'])+"_"+str(row_dict['Second Object Number'])
        #objnum=str(row_dict['First Image Number'])+"_"+str(row_dict['First Object Number'])
        
        cell_key_word="Second"
        obj_key_word="First"
        objtype=row_dict['First Object Name']
    
    wellid = imgnum_to_well[image_ID]
    
    cell_obj_id = str(row_dict[cell_key_word+' Object Number'])
    obj_obj_id = str(row_dict[obj_key_word+' Object Number'])
    
    cellnum=wellid+"_"+cell_obj_id
    objnum=wellid+"_"+obj_obj_id
    
    if objnum+objtype not in done:



        if objtype == "Red_cytoplasm":
            intensity = ApoptosisObjects_dict[objnum]["intensity"]
            area = ApoptosisObjects_dict[objnum]["area"]
        elif objtype == "Transfected":
            intensity = TFObjects_dict[objnum]["intensity"]
            area = TFObjects_dict[objnum]["area"]
        Cell_dict[cellnum][objtype]["list of vals"].append([intensity,area]) 
        
        done.add(objnum+objtype)

  0%|          | 0/2313096 [00:00<?, ?it/s]

In [12]:
#FUNCTION TO NORMALIZE SIGNALS TO SIGNAL AREA
def get_intensity(list_of_vals):
    intensity_sum = 0
    area_sum = 0
    for row in list_of_vals:
        intensity_sum += float(row[0])
        area_sum += float(row[1])
        
    if list_of_vals == []:
        return 0
    
    return float(intensity_sum)/float(area_sum)

In [13]:
#EXPORT INDIVIDUAL CELL VALUES AS A CONTROL (MULTIPLE PER WELL)
export1 = []
header1 = ["cell ID","well ID","Apoptosis signal","Transfection signal"]
export1.append(header1)

for cell in Cell_dict:
    Cell_dict[cell]["Red_cytoplasm"]["val"] = get_intensity(Cell_dict[cell]["Red_cytoplasm"]["list of vals"])
    Cell_dict[cell]["Transfected"]["val"] = get_intensity(Cell_dict[cell]["Transfected"]["list of vals"])
    
    next_line = [cell,cell.split("_")[0],Cell_dict[cell]["Red_cytoplasm"]["val"],Cell_dict[cell]["Transfected"]["val"]]
    export1.append(next_line)

In [14]:
#EXPORT FILE WRITING FUNCTION
def the_saver(output_path, list_of_lists):
    with open(output_path,'w', newline='',encoding='utf-8-sig',errors='ignore') as csvfile:
        for row in list_of_lists:
            csv.writer(csvfile, dialect='excel').writerow(row)

In [15]:
the_saver(outpath1, export1)

In [16]:
#EXPORT APOPTOSIS SIGNAL PER WELL ONLY FOR TRANSFECTED CELLS

#TRANSFECTION THRESHOLD
min_TF_sig = 0

well_signal_dict = {}

for row in export1[1:]:
    well_id = row[1]
    apop_sig = float(row[2])
    TF_sig = float(row[3])
    
    if TF_sig > min_TF_sig:
        
        if well_id not in well_signal_dict:
            well_signal_dict[well_id] = []
            
        well_signal_dict[well_id].append(apop_sig)
        
        

In [17]:
export2 = []
header2 = ["well ID","Apoptosis signal","no of cells"]
export2.append(header2)

for well in well_signal_dict:
    num_cells = len(well_signal_dict[well])
    average = np.mean(well_signal_dict[well])
    
    next_line = [well,average,num_cells]
    export2.append(next_line)
    
the_saver(outpath2, export2)

In [18]:
#EXPORT ALL APOPTOSIS SIGNAL PER WELL

#TRANSFECTION THRESHOLD
min_TF_sig = -1

well_signal_dict = {}

for row in export1[1:]:
    well_id = row[1]
    apop_sig = float(row[2])
    TF_sig = float(row[3])
    
    if TF_sig > min_TF_sig:
        
        if well_id not in well_signal_dict:
            well_signal_dict[well_id] = []
            
        well_signal_dict[well_id].append(apop_sig)
        
        

In [19]:
export3 = []
header3 = ["well ID","Apoptosis signal","no of cells"]
export3.append(header3)

for well in well_signal_dict:
    num_cells = len(well_signal_dict[well])
    average = np.mean(well_signal_dict[well])
    
    next_line = [well,average,num_cells]
    export2.append(next_line)
    
the_saver(outpath3, export3)